In [1]:
import os
import nrrd
import numpy as np
from tqdm import tqdm
import matplotlib.pyplot as plt

In [2]:
directory = "#####/PDDCA_data/PDDCA-1.4.1"

positive = []
negative = []
# log where are we loading the data from

count = 0
# all the patients are facing left
for patient_path in tqdm(os.listdir(directory)):
    
    if patient_path.startswith('.'):
        continue

    # get paths where things are stored
    brain_image_path = os.path.join(directory, patient_path, 'img.nrrd')
    optic_left_path = os.path.join(directory, patient_path, 'structures/OpticNerve_L.nrrd')
    optic_right_path = os.path.join(directory, patient_path, 'structures/OpticNerve_R.nrrd')

    # load the nrrd images
    brain_image, _ = nrrd.read(brain_image_path)
    optic_left, _ = nrrd.read(optic_left_path)
    optic_right, _ = nrrd.read(optic_right_path)

    # get left and right optic nerves togehter
    optic = optic_left + optic_right

    # put them in the output format
    for i in range(brain_image.shape[-1]):

        mask = optic[:,:,i]
        data_dicom = brain_image[:,:,i]


        # windowing brain image
        img_min = np.amin(data_dicom)
        img_max = np.amax(data_dicom)

        window_width = 300
        window_center = 36
                
        img_min = window_center - window_width//2 #minimum HU level
        img_max = window_center + window_width//2 #maximum HU level
        data_dicom[data_dicom<img_min] = img_min #set img_min for all HU levels less than minimum HU level
        data_dicom[data_dicom>img_max] = img_max

        data_dicom = (((data_dicom - img_min) * 255) / (img_max-img_min)) + 0

        # stacked image
        stacked_image = np.zeros((512, 512, 3), dtype=np.uint8)
        stacked_image[:, :, 0] = data_dicom - (mask * data_dicom)  + mask * 255  # Blue channel (grayscale image1)
        stacked_image[:, :, 1] = data_dicom - (mask * data_dicom)  # Green channel (grayscale image1)
        stacked_image[:, :, 2] = data_dicom - (mask * data_dicom)  # Red channel (grayscale image2)

        # rotate stacked image
        stacked_image = np.rot90(stacked_image, k=-1)
        data_dicom = np.rot90(data_dicom, k=-1)

        if optic[:,:,i].any():
            plt.imshow(data_dicom, cmap="bone")
            plt.savefig(f"positive/patient{patient_path}_slice{i}.png", dpi=800)
            plt.close()
            plt.imshow(stacked_image, cmap="bone")
            plt.savefig(f"positive/patient{patient_path}_slice{i}_label.png", dpi=800)
            plt.close()
        else:
            plt.imshow(data_dicom, cmap="bone")
            plt.savefig(f"negative/patient{patient_path}_slice{i}.png", dpi=800)
            plt.close()

    count += 1

    if count == 20:
        break

 41%|████      | 20/49 [1:09:16<1:40:27, 207.83s/it]
